# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [32]:
import pandas as pd                                                              # Load datasets
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [30]:
# create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000
condition1 = df["Total Claim Amount"] >1000            #create the condition1
condition2 = df["Response"] == "Yes"                   #create the condition2
new_df = df[condition1 & condition2]                   #combined them
new_df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [41]:
condition2 = df["Response"] == "Yes"                   #create the condition2
filtered_by_resp_df = df[condition2]  
pivot_df = filtered_by_resp_df.pivot_table(index='Policy Type', columns='Gender', values=['Total Claim Amount'], aggfunc='mean')
round(pivot_df,2)

Total Claim Amount        
Gender                          F       M
Policy Type                              
Corporate Auto             433.74  408.58
Personal Auto              452.97  457.01
Special Auto               453.28  429.53

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [61]:
#grup by State and counts the unique Cuntomer
cunstomer_counts = df.groupby("State")["Customer"].nunique()
cunstomer_counts

State
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: Customer, dtype: int64

In [65]:
#filter the results to include only states where there are more than 500 customers.
filtered_states = cunstomer_counts[cunstomer_counts >500]
filtered_states

State
Arizona       1703
California    3150
Nevada         882
Oregon        2601
Washington     798
Name: Customer, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [56]:
#maximum of customer lifetime value by education level and gender 
pivot_df1 = df.pivot_table(index='Education', columns='Gender', values=['Customer Lifetime Value'], aggfunc='max')
round(pivot_df1,2)

Customer Lifetime Value          
Gender                                     F         M
Education                                             
Bachelor                            73225.96  67907.27
College                             61850.19  61134.68
Doctor                              44856.11  32677.34
High School or Below                55277.45  83325.38
Master                              51016.07  50568.26

In [57]:
#minimum of customer lifetime value by education level and gender 
pivot_df2 = df.pivot_table(index='Education', columns='Gender', values=['Customer Lifetime Value'], aggfunc='min')
round(pivot_df2,2)

Customer Lifetime Value         
Gender                                     F        M
Education                                            
Bachelor                             1904.00  1898.01
College                              1898.68  1918.12
Doctor                               2395.57  2267.60
High School or Below                 2144.92  1940.98
Master                               2417.78  2272.31

In [58]:
#median of customer lifetime value by education level and gender 
pivot_df3 = df.pivot_table(index='Education', columns='Gender', values=['Customer Lifetime Value'], aggfunc='median')
round(pivot_df3,2)

Customer Lifetime Value         
Gender                                     F        M
Education                                            
Bachelor                             5640.51  5548.03
College                              5623.61  6005.85
Doctor                               5332.46  5577.67
High School or Below                 6039.55  6286.73
Master                               5729.86  5579.10

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here